In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
plotly.tools.set_credentials_file(username='vvr', api_key='Ker155UU70wk3Ccz6iyM')
import plotly.plotly as py
import plotly.graph_objs as go

color = sns.color_palette("husl", 8)
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


%matplotlib inline

import squarify

import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
#from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
import re

In [2]:
df= pd.read_csv('train.csv')

In [3]:
df['project_essay_3'].fillna('unknown',inplace = True)
df['project_essay_4'].fillna('unknown',inplace = True)

#df.sample()

Now with the Subject SubCategories

Dealing with Project Title

In [4]:
df['project_title'] = df['project_title'].apply(lambda x: " ".join(x.lower() for x in x.split()))

df['project_title'] = df['project_title'].map(lambda x: re.sub(r'\W+', ' ', x))


In [5]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

from nltk.corpus import stopwords
stop = stopwords.words('english')

df['project_title'] = df.apply(lambda row: nltk.word_tokenize(row['project_title']), axis=1)

df['project_title'] = df["project_title"].apply(lambda x: [stemmer.stem(y) for y in x])

df['project_title'] = df['project_title'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))

In [6]:
df['project_title'] = df['project_title'].apply(lambda x: " ".join(x if x.isalpha() else " " for x in x.split()))


Adding Word Counts

In [7]:
df['count_1'] = df['project_essay_1'].apply(lambda x : len(str(x).split(' ')))

df['count_2'] = df['project_essay_2'].apply(lambda x : len(str(x).split(' ')))

df['count_3'] = df['project_essay_3'].apply(lambda x : len(str(x).split(' ')))
df['count_4'] = df['project_essay_4'].apply(lambda x : len(str(x).split(' ')))
df['count_s'] = df['project_resource_summary'].apply(lambda x : len(str(x).split(' ')))

Dealing with project_essay_1

In [8]:
import re

df['project_essay_1'] = df['project_essay_1'].map(lambda x: re.sub(r'\W+', ' ', x))
df['project_essay_1'] = df['project_essay_1'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['project_essay_1'] = df['project_essay_1'].str.findall('\w{3,}').str.join(' ')
#df['project_essay_1'] = df['project_essay_1'].str.findall('\w{3,}').str.join(' ')

In [9]:
import nltk
df['project_essay_1'] = df.apply(lambda row: nltk.word_tokenize(row['project_essay_1']), axis=1)

In [10]:
df['project_essay_1'] = df["project_essay_1"].apply(lambda x: [stemmer.stem(y) for y in x])


In [11]:
df['project_essay_1'] = df['project_essay_1'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))


We will do the Project essay2

In [12]:

df['project_essay_2'] = df['project_essay_2'].map(lambda x: re.sub(r'\W+', ' ', x))
df['project_essay_2'] = df['project_essay_2'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['project_essay_2'] = df['project_essay_2'].str.findall('\w{3,}').str.join(' ')
#df['project_essay_1'] = df['project_essay_1'].str.findall('\w{3,}').str.join(' ')

df['project_essay_2'] = df.apply(lambda row: nltk.word_tokenize(row['project_essay_2']), axis=1)

df['project_essay_2'] = df["project_essay_2"].apply(lambda x: [stemmer.stem(y) for y in x])

df['project_essay_2'] = df['project_essay_2'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))


In [13]:
df['project_submitted_datetime'] = pd.to_datetime(df['project_submitted_datetime']) 
df['project_submitted_datetime'] = df['project_submitted_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))


In [14]:
from pandas import Series

df['new_date_int'] = Series(df['project_submitted_datetime'], dtype='datetime64')
df['new_date_int'] = df['new_date_int'].view('int64')

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\internals.py:3224: FutureWarning:

Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.



# Label Encoder

In [15]:
# Label encoding
#copied
#thanks
#https://www.kaggle.com/jmbull/xtra-credit-xgb-lgb-w-tfidf-feature-stacking


cols = [
    'teacher_id', 
    'teacher_prefix', 
    'school_state', 
    'project_grade_category', 
    'project_subject_categories', 
    'project_subject_subcategories'
]

for c in (cols):
    le = LabelEncoder()
    le.fit(df[c].astype(str))
    
    df[c] = le.transform(df[c].astype(str))
    #X_test[c] = le.transform(X_test[c].astype(str))
    


In [16]:
X = df[['teacher_id', 'teacher_prefix', 'school_state',
      'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects',
       'count_1', 'count_2', 'count_3', 'count_4', 'count_s', 'new_date_int']].copy()

In [17]:
y = df['project_is_approved'].copy()

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Tfidf

In [19]:
X_train.sample()

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,count_1,count_2,count_3,count_4,count_s,new_date_int
155536,90251,1,9,0,32,18,fifth grade want work digit world,typic day classroom imagin town center see fif...,incorpor lesson involv technolog daili class r...,unknown,unknown,My students need tech-savvy classroom for thei...,1,123,224,1,1,19,1470096000000000000


In [20]:
X_train['essay1'] = X_train['project_essay_1'] + X_train['project_essay_2']
X_test['essay1'] = X_test['project_essay_1'] + X_test['project_essay_2']

X_test['essay2'] = X_test['project_essay_3'] + X_test['project_essay_4']
X_train['essay2'] = X_train['project_essay_3'] + X_train['project_essay_4']

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [21]:
X_train.columns

Index(['teacher_id', 'teacher_prefix', 'school_state',
       'project_grade_category', 'project_subject_categories',
       'project_subject_subcategories', 'project_title', 'project_essay_1',
       'project_essay_2', 'project_essay_3', 'project_essay_4',
       'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'count_1', 'count_2',
       'count_3', 'count_4', 'count_s', 'new_date_int', 'essay1', 'essay2'],
      dtype='object')

In [22]:
X_train.drop(['project_essay_1',
       'project_essay_2', 'project_essay_3', 'project_essay_4'], axis =1, inplace = True)

X_test.drop(['project_essay_1',
       'project_essay_2', 'project_essay_3', 'project_essay_4'], axis =1,inplace = True)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [23]:
X_train.sample()

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_resource_summary,teacher_number_of_previously_posted_projects,count_1,count_2,count_3,count_4,count_s,new_date_int,essay1,essay2
133473,92049,3,33,0,32,194,technolog provid potenti,"My students need new, updated and unbroken tec...",0,175,151,1,1,11,1488585600000000000,teach year old probabl one challeng yet reward...,unknownunknown


# Count Vectorizer

In [24]:
X_train.sample()

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_resource_summary,teacher_number_of_previously_posted_projects,count_1,count_2,count_3,count_4,count_s,new_date_int,essay1,essay2
165817,79966,2,10,3,32,199,chrome reflect,"My students need a Chromebook to research, dev...",0,137,139,1,1,14,1477267200000000000,student eager learn enjoy type activ classroom...,unknownunknown


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

data = ['essay1', 'essay2', 'project_title','project_resource_summary',]
'''
for i in (data):
    count_vect = CountVectorizer()
    X_train[i] = count_vect.fit_transform(X_train[i])
    X_test[i] = count_vect.fit_transform(X_test[i])
'''

'\nfor i in (data):\n    count_vect = CountVectorizer()\n    X_train[i] = count_vect.fit_transform(X_train[i])\n    X_test[i] = count_vect.fit_transform(X_test[i])\n'

In [30]:
X_train.head()

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_resource_summary,teacher_number_of_previously_posted_projects,count_1,count_2,count_3,count_4,count_s,new_date_int,essay1,essay2
62080,6869,2,27,1,32,282,solv problem use scienc,My students need t these science supplies in o...,85,148,307,1,1,23,1466899200000000000,"(0, 32188)\t0.05187802603715875\n (0, 26701...",unknownunknown
106954,77692,2,10,3,28,323,love read,My students need books for the classroom. The...,0,103,110,1,1,28,1478476800000000000,"(0, 32188)\t0.05187802603715875\n (0, 26701...",unknownunknown
62773,58899,2,15,0,24,334,let make book,My students need a binding machine to make the...,23,121,104,1,1,13,1477699200000000000,"(0, 32188)\t0.05187802603715875\n (0, 26701...",unknownunknown
170134,76234,1,17,2,40,361,crash band boom need cymbal stand,My students need cymbal stands so we can use w...,9,146,227,1,1,18,1490832000000000000,"(0, 32188)\t0.05187802603715875\n (0, 26701...",unknownunknown
168145,49617,2,43,3,28,335,technolog classroom,My students need ipads to continue to improve ...,4,84,103,1,1,12,1488412800000000000,"(0, 32188)\t0.05187802603715875\n (0, 26701...",unknownunknown


In [31]:
X_train['essay1'].toarray()

AttributeError: 'Series' object has no attribute 'toarray'